# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 24 2023 11:10AM,258128,12,HH9018,Hush Hush,Released
1,Feb 24 2023 10:59AM,258127,10,0086430510,ISDIN Corporation,Released
2,Feb 24 2023 10:59AM,258127,10,0086430560,ISDIN Corporation,Released
3,Feb 24 2023 10:56AM,258125,10,0086428404,ISDIN Corporation,Released
4,Feb 24 2023 10:56AM,258125,10,0086428410,ISDIN Corporation,Released
5,Feb 24 2023 10:56AM,258125,10,0086428411,ISDIN Corporation,Released
6,Feb 24 2023 10:56AM,258125,10,0086428431,ISDIN Corporation,Released
7,Feb 24 2023 10:56AM,258125,10,0086428433,ISDIN Corporation,Released
8,Feb 24 2023 10:56AM,258125,10,0086428437,ISDIN Corporation,Released
9,Feb 24 2023 10:56AM,258125,10,0086428435,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,258123,Released,2
33,258124,Released,1
34,258125,Released,11
35,258127,Released,2
36,258128,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
258123,NaN,2.0
258124,NaN,1.0
258125,NaN,11.0
258127,NaN,2.0
258128,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
258044,1.0,0.0
258052,0.0,1.0
258053,6.0,4.0
258058,0.0,48.0
258061,0.0,15.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
258044,1,0
258052,0,1
258053,6,4
258058,0,48
258061,0,15


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,258044,1,0
1,258052,0,1
2,258053,6,4
3,258058,0,48
4,258061,0,15


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,258044,1,
1,258052,,1
2,258053,6,4
3,258058,,48
4,258061,,15


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 24 2023 11:10AM,258128,12,Hush Hush
1,Feb 24 2023 10:59AM,258127,10,ISDIN Corporation
3,Feb 24 2023 10:56AM,258125,10,ISDIN Corporation
14,Feb 24 2023 10:53AM,258124,16,"SHL Pharma, LLC"
15,Feb 24 2023 10:46AM,258123,10,ISDIN Corporation
17,Feb 24 2023 10:41AM,258122,10,ISDIN Corporation
42,Feb 24 2023 10:16AM,258121,16,Sartorius Bioprocess Solutions
43,Feb 24 2023 10:14AM,258120,19,"GUSA Granules USA, Inc."
44,Feb 24 2023 9:51AM,258118,10,Eywa Pharma Inc.
46,Feb 24 2023 9:47AM,258119,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Feb 24 2023 11:10AM,258128,12,Hush Hush,,1
1,Feb 24 2023 10:59AM,258127,10,ISDIN Corporation,,2
2,Feb 24 2023 10:56AM,258125,10,ISDIN Corporation,,11
3,Feb 24 2023 10:53AM,258124,16,"SHL Pharma, LLC",,1
4,Feb 24 2023 10:46AM,258123,10,ISDIN Corporation,,2
5,Feb 24 2023 10:41AM,258122,10,ISDIN Corporation,,25
6,Feb 24 2023 10:16AM,258121,16,Sartorius Bioprocess Solutions,,1
7,Feb 24 2023 10:14AM,258120,19,"GUSA Granules USA, Inc.",,1
8,Feb 24 2023 9:51AM,258118,10,Eywa Pharma Inc.,,2
9,Feb 24 2023 9:47AM,258119,10,"Citieffe, Inc.",1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 24 2023 11:10AM,258128,12,Hush Hush,1,
1,Feb 24 2023 10:59AM,258127,10,ISDIN Corporation,2,
2,Feb 24 2023 10:56AM,258125,10,ISDIN Corporation,11,
3,Feb 24 2023 10:53AM,258124,16,"SHL Pharma, LLC",1,
4,Feb 24 2023 10:46AM,258123,10,ISDIN Corporation,2,
5,Feb 24 2023 10:41AM,258122,10,ISDIN Corporation,25,
6,Feb 24 2023 10:16AM,258121,16,Sartorius Bioprocess Solutions,1,
7,Feb 24 2023 10:14AM,258120,19,"GUSA Granules USA, Inc.",1,
8,Feb 24 2023 9:51AM,258118,10,Eywa Pharma Inc.,2,
9,Feb 24 2023 9:47AM,258119,10,"Citieffe, Inc.",,1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 24 2023 11:10AM,258128,12,Hush Hush,1,
1,Feb 24 2023 10:59AM,258127,10,ISDIN Corporation,2,
2,Feb 24 2023 10:56AM,258125,10,ISDIN Corporation,11,
3,Feb 24 2023 10:53AM,258124,16,"SHL Pharma, LLC",1,
4,Feb 24 2023 10:46AM,258123,10,ISDIN Corporation,2,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Feb 24 2023 11:10AM,258128,12,Hush Hush,1.0,NaN
1,Feb 24 2023 10:59AM,258127,10,ISDIN Corporation,2.0,NaN
2,Feb 24 2023 10:56AM,258125,10,ISDIN Corporation,11.0,NaN
3,Feb 24 2023 10:53AM,258124,16,"SHL Pharma, LLC",1.0,NaN
4,Feb 24 2023 10:46AM,258123,10,ISDIN Corporation,2.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 24 2023 11:10AM,258128,12,Hush Hush,1.0,0.0
1,Feb 24 2023 10:59AM,258127,10,ISDIN Corporation,2.0,0.0
2,Feb 24 2023 10:56AM,258125,10,ISDIN Corporation,11.0,0.0
3,Feb 24 2023 10:53AM,258124,16,"SHL Pharma, LLC",1.0,0.0
4,Feb 24 2023 10:46AM,258123,10,ISDIN Corporation,2.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,4903743,209.0,7.0
12,516172,1.0,1.0
15,774266,10.0,8.0
16,516245,2.0,0.0
19,1290464,17.0,41.0
20,258052,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,4903743,209.0,7.0
1,12,516172,1.0,1.0
2,15,774266,10.0,8.0
3,16,516245,2.0,0.0
4,19,1290464,17.0,41.0
5,20,258052,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,209.0,7.0
1,12,1.0,1.0
2,15,10.0,8.0
3,16,2.0,0.0
4,19,17.0,41.0
5,20,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,209.0
1,12,Released,1.0
2,15,Released,10.0
3,16,Released,2.0
4,19,Released,17.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20
Status,,,,,,
Executing,7.0,1.0,8.0,0.0,41.0,0.0
Released,209.0,1.0,10.0,2.0,17.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20
0,Executing,7.0,1.0,8.0,0.0,41.0,0.0
1,Released,209.0,1.0,10.0,2.0,17.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20
0,Executing,7.0,1.0,8.0,0.0,41.0,0.0
1,Released,209.0,1.0,10.0,2.0,17.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()